In [ ]:
import pandas as pd
import os 
import csv
import sqlalchemy
from sqlalchemy.engine import url as sa_url
import pprint
import datacompy as dcp
import numpy as np
import pprint
from pandas.api.types import is_datetime64_ns_dtype, is_numeric_dtype

In [ ]:
sql_password_rs='' # input password for fi_etl
sql_password_mysql='' # input password for fi_etl

In [ ]:
db_connect_url_rs = sa_url.URL(drivername='postgresql',
                                 username='fi_etl',
                                 password=sql_password_rs,
                                 host='bi-redshift.clfukyskh3cy.eu-west-1.redshift.amazonaws.com',
                                 port=5439,
                                 database='bi_db')

In [ ]:
db_connect_url_mysql = sa_url.URL(drivername='mysql+pymysql',
                                 username='fi_etl',
                                 password=sql_password_mysql,
                                 host='aws-eu-bi-rds-mysql-prod.c4kjlqvxj0sw.eu-west-1.rds.amazonaws.com',
                                 port=21599,
                                 database='wkda')

In [ ]:
db_conn_mysql = sqlalchemy.create_engine(db_connect_url_mysql).connect()
db_conn_rs = sqlalchemy.create_engine(db_connect_url_rs).connect()


In [ ]:
mysql="select * from dwh_load.claims_weekly"

In [ ]:
rs="select * from dwh_fi.claims_weekly"

In [ ]:
rs = pd.read_sql_query(sql=rs, con=db_conn_rs)

In [ ]:
mysql = pd.read_sql_query(sql=mysql, con=db_conn_mysql)

In [ ]:
# use datacompy for comparison chek 
# join columns parameter given on the most granular id of the datasets
compare = dcp.Compare(
    rs,
    mysql,
    join_columns=['claim_number','claim_reason_id', 'claim_evaluation_id', 'decision_id'],  #You can also specify a list of columns
    abs_tol=0, #Optional, defaults to 0
    rel_tol=0, #ptional, defaults to 0
    df1_name='df_rs', #Optional, defaults to 'df1'
    df2_name='df_mysql' #Optional, defaults to 'df2'
    )

In [ ]:
In below results check : 
    It must be Any duplicates on match values: No , otherwise check again your join columns 
    Number of rows with some compared columns unequal: 779,923
    Number of rows with all compared columns equal: 0

In [ ]:
print(compare.report())

From the results of compare.report() above we see that all mutual rows are unequal 779923 rows 
timestamp columns are not equal because of some digits after seconds part in redshift  
and float columns are unequal because of digits also 
Use method below to correct those and re-run datacompy 

In [ ]:
    def pandas_manipulate(data):
        """
        This method will remove extra digits after seconds from columns in the pandas dataframe that have 'date' or '_on' in them
        so it will convert value 2021-07-26 09:29:04+00:00 to 2021-07-26 09:29:04
        and also will convert float64 data type columns into integer columns , to discard decimals
        :param data: Dataframe object to be manipulated
        :return: the changed pandas DataFrame object
        """
        # find date columns by string searching column names containing '_on' and 'date'
        time_cols = [col for col in data.columns if '_on' in col]

        # find float64 type columns
        float_cols = list(data.select_dtypes(include=[np.float64]).columns)

        # iterate over date selected columns of the pd dataframe to remove digits after seconds
        for col in data[time_cols]:
            data[col] = data[col].astype('datetime64[s]')

        # iterate over float type selected columns of the pd dataframe to convert float64 into int
        for col in data[float_cols]:
            # fill na values from amount columns with a preset big integer like 1999999
            data[col] = data[col].fillna(1999999)
            # convert those amount columns into integers
            data[col] = data[col].astype(int)
            # convert back to na the rows that got the preset 1999999 value
            data.loc[data[col] == 1999999, col] = np.nan

        return data

In [ ]:
rs1=pandas_manipulate(rs)
mysql1=pandas_manipulate(mysql)

In [ ]:
df_rs1=rs1[['claim_number','claim_reason_id', 'claim_evaluation_id', 'decision_id', 'reason_created_on']]


In [ ]:
df_mysql1=mysql1[['claim_number','claim_reason_id', 'claim_evaluation_id', 'decision_id', 'reason_created_on']]

In [ ]:
# new datacompy run with altered dataframes 
compare = dcp.Compare(
    df_rs1,
    df_mysql1,
    join_columns=['claim_number','claim_reason_id', 'claim_evaluation_id', 'decision_id'],  #You can also specify a list of columns
    abs_tol=0, #Optional, defaults to 0
    rel_tol=0, #ptional, defaults to 0
    df1_name='df_rs', #Optional, defaults to 'df1'
    df2_name='df_mysql' #Optional, defaults to 'df2'
    )

In [ ]:
print(compare.report())

From above we see that we still have all rows as unequal
However this does not reflect the truth. If only 1 column exists with all values unequal 
then above check will show that datasets are unequal on all rows . It is enough for 1 column to be unequal in all rows
for a row to be considered unequal in all columns. 
This can be tackled if we compare subsets of the original datasets 
on join columns and on columns of interest of our choice . Those columns of interest will not be checked altogether in 1 check 
but will be checked one by one, meaning that if we specify 3 columns of interest , method below will subset the initial dataset 1 into 3 subests , each having one of those columns. Then equivalent subsets from dataset 2 will be created and compared against subsets of dataset 1 .
This way we can know if a column is indeed equal to the other , without the result being affected by other columns' inequality.

Timestamp columns might not be of interest, because we know that MySQL and Redshift have a replication lag of at least 3 hours 
Also we do not care if 2 timestamps differ some minutes between them .

In [ ]:
# method to iterate over the elements of a list 2 by 2 
# test with lst=[1,2,3,4] by calling grouped(lst,2)
def grouped(iterable, n):
    "s -> (s0,s1,s2,...sn-1), (sn,sn+1,sn+2,...s2n-1), (s2n,s2n+1,s2n+2,...s3n-1), ..."
    return zip(*[iter(iterable)]*n)

In [ ]:
def comparison_one_by_one_columns(df_rs, df_mysql, columns: list, join_columns: list): 
    """
    This method will compare 2 subsets of 2 initial datasets, based on the column names specified 
    in columns list object . Comparison is performed on each column independently of other columns in list.
    For example passing parameter columns=['col1', 'col2', 'col3'] ,the subset datasets compared in datacompy 
    from the initial datasets contain columns 
    ['join_column_1', 'join_column_2', ... 'join_column_n', 'col1'], ['join_column_1', 'join_column_2', ... 'join_column_n', 'col2']  etc etc. 
    :param df_rs: the Redshift pandas dataframe , or any dataset 
    :param df_mysql: the MySQL pandas dataframe , or any dataset 
    :param columns: the list of columns upon which the equality checks will be performed 
    :param join_columns: the list of columns to be used in join_columns parameter of datacompy.Compare object
    :return: dict :dictionary object storing key = elements in columns list , value = number of matching rows for that column from the comparison
             comp : list object storing the compare objects 
    """
    # list to contain pandas dataframe objects from slicing intitial dataframes 
    # from redshift and mysql with the 1 column to be checked for equality and the join columns 
    dfs=[]
    for col in columns: 
        colm=join_columns + list(col.split(" "))
        dfs.append(df_rs.loc[:,colm])
        dfs.append(df_mysql.loc[:,colm])
    # list to contain compare objects from comparing 2 by 2 the pandas dataframes stored in dfs
    comp=[]
    for x, y in grouped(dfs, 2):
        compare = dcp.Compare(
        x,
        y,
        join_columns=join_columns,  #You can also specify a list of columns
        abs_tol=0, #Optional, defaults to 0
        rel_tol=0, #ptional, defaults to 0
        df1_name='df_rs', #Optional, defaults to 'df1'
        df2_name='df_mysql' #Optional, defaults to 'df2'
        )
        comp.append(compare)
    dict={}
    for (a,b) in zip(columns,comp): 
        dict[a]=b.count_matching_rows()
    return  dict,comp

In [ ]:
# run the method above 
dict,comp =comparison_one_by_one_columns(rs1, mysql1, ['stock_number','vin', 'merchant_id','company','merchant_country', 'merchant_score', 'merchant_score_category', 'account_score_category', 'claim_status', 'claim_status_id', 'fault_by', 'evaluation_amount', 'crd_credit', 'crd_auc_vouchers', 'crd_partial_refund', 'crd_full_refund', 'crd_no_comp', 'crd_free_transport', 'type', 'transport_comp', 'crd_vouchers','crd_vouchers_value', 'category_desc', 'sub_category_desc', 'part_desc', 'dmg_desc', 'branch_name', 'branch_id','sell_price', 'buy_price', 'purchase_country', 'car_mileage', 'built_year', 'evaluator_name', 'claim_created_on', 'reason_created_on'], ['claim_number','claim_reason_id', 'claim_evaluation_id', 'decision_id'])

In [ ]:
# print method results 
for k, v in dict.items():
    print(k, v)

From above we see that the vast majority of columns in the 2 datasets are 100 % equal, 
the results from the first 2 checks showed that all mutual rows were unequal because potentially one of the timestamp columns 
was not matching at all , however all other columns are matching 100% 
Find out which of the timestamp columns were causing this 

In [ ]:
dict,comp =comparison_one_by_one_columns(rs1, mysql1, ['claim_created_on','claim_last_modified_on','reason_created_on', 'first_registration_date', 'dwh_created_on', 'dwh_modified_on'], ['claim_number','claim_reason_id', 'claim_evaluation_id', 'decision_id'])

In [ ]:
rs1.dtypes

In [ ]:
# print method results 
for k, v in dict.items():
    print(k, v)

So 2 columns are causing all this. dwh_created_on, dwh_modified_on mismatch 100% 
Those columns have 1 distinct value , and the reason those are different is beacuse of the replication lag between mysql and redshift. 
Run below to see in more detail the compare report of one of the elements in comp list 

In [ ]:
print(comp[3].report())

In [ ]:
# chack ALL columns now to finish the analysis
dict,comp =comparison_one_by_one_columns(rs1, mysql1, ['built_year','sell_price','crd_vouchers_value','crd_vouchers','transport_comp','type','crd_free_transport','crd_no_comp','crd_full_refund','crd_partial_refund','crd_auc_vouchers','crd_credit','evaluation_amount','claim_status_id','dwh_report_desc','dwh_report_id','evaluator_name','purchase_country','buy_price','branch_name','dmg_desc','part_desc','sub_category_desc','category_desc','fault_by','claim_status','account_score_category','merchant_score_category','merchant_country','company','vin','stock_number','id','first_registration_date','car_mileage','branch_id','merchant_id','merchant_score','dwh_modified_on','dwh_created_on','reason_created_on','claim_last_modified_on','claim_created_on'], ['claim_number','claim_reason_id', 'claim_evaluation_id', 'decision_id'])

In [ ]:
# print method results 
for k, v in dict.items():
    print(k, v)

In [ ]:
def timestamp_cols_max_diff(df1: pd.DataFrame, df2: pd.DataFrame, cols: list, join_cols: list): 
    """
    Method will calculate the absolute difference in seconds for datetime64[ns] data type columns ,
    over 2 pandas dataframes for rows of matching columns specified 
    and will return a dictionary object with this max difference 
    :param df1:The first pandas dataframe we want to compare with another 
    :param df2:The second pandas dataframe we want to compare with the first 
    :param cols: A list of datetime64[ns] type columns that we want to calculate their difference in seconds 
    :param join_cols: A list of columns to join df1 and df2
    :return: A dictionary with key= the columns in cols and value=the max difference found in seconds between 
            rows of the 2 datasets
    """
    error_cols=[]
    for col in cols: 
        if not (is_datetime64_ns_dtype(df1[col]) and  is_datetime64_ns_dtype(df2[col])):
            error_cols.append(col)
    if error_cols:
        print(f"Columns are not datetime64-ns data type : {error_cols}")
    else:
        print("All columns have the correct data type")
    m = pd.merge(df1, df2, how='inner', on= join_cols)
    dict={}
    for col in cols: 
        m[col+'_diff']=abs(m[col+'_x'] - m[col+'_y'])
        dict[col+'_max_diff']=max(m[col+'_diff'])
    return dict    

In [ ]:
gg=timestamp_cols_max_diff(rs1,mysql1,['claim_created_on', 'claim_last_modified_on','reason_created_on'], ['claim_number','claim_reason_id', 'claim_evaluation_id', 'decision_id'] )

In [ ]:
# print method results 
for k, v in gg.items():
    print(k, v)

In [ ]:
rs1.dtypes

In [ ]:
def float_int_cols_max_diff(df1: pd.DataFrame, df2: pd.DataFrame, cols: list, join_cols: list): 
    """
    Method will calculate the absolute difference between int64 or float64 data type columns ,
    over 2 pandas dataframes for rows of matching columns specified 
    and will return a dictionary object with this max difference 
    :param df1:The first pandas dataframe we want to compare with another 
    :param df2:The second pandas dataframe we want to compare with the first 
    :param cols: A list of int64 or float64 type columns that we want to calculate their difference
    :param join_cols: A list of columns to join df1 and df2
    :return: A dictionary with key= the columns in cols and value=the max difference found in seconds between 
            rows of the 2 datasets
    """
    error_cols=[]
    for col in cols: 
        if not (is_numeric_dtype(df1[col]) and  is_numeric_dtype(df2[col])):
            error_cols.append(col)
    if error_cols:
        print(f"Columns are not float64 or int64 data type : {error_cols}")
    else:
        print("All columns have the correct data type")
    m = pd.merge(df1, df2, how='inner', on= join_cols)
    dict={}
    for col in cols: 
        m[col+'_diff']=abs(m[col+'_x'] - m[col+'_y'])
        dict[col+'_max_diff']=max(m[col+'_diff'])
    return dict    

In [ ]:
ff=float_int_cols_max_diff(rs1,mysql1,['car_mileage','evaluation_amount'],['claim_number','claim_reason_id', 'claim_evaluation_id', 'decision_id'] )

In [ ]:
# print method results 
for k, v in ff.items():
    print(k, v)

Extract dataset from comparison object of interest results locally for more drill down 
For example here  say that we are interested in column claim_created_on , so this will reside in comp[35]

In [ ]:
df = pd.DataFrame(comp[35].intersect_rows)


In [ ]:
df.to_csv('comparison_object.csv', sep=';', index=False)